# Part I. ETL Pipeline for Pre-Processing the Files

## Run the code below for preprocessing the files

#### Import Python packages 

In [13]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [14]:
# check the current working directory
print(os.getcwd())

# Get the current folder and subfolder event_data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

c:\Users\ofili\Documents\Projects\Data Engineering\Data Modeling with Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [15]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Get total number of rows 
#print(len(full_data_rows_list))

# Get the list of event_data rows
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [16]:
# check the number of rows in the new csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Load data into cassandra database tables 

The CSV file is ready for querying. The file is located within the Workspace directory and named <font color=red>**event_datafile_new.csv**</font>. 

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [17]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [18]:
# Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [20]:
#Set KEYSPACE to the keyspace specified above

session.set_keyspace('music')

## Create queries to ask the following three questions of the data

For this project, we look at three scenarios. For each scenario, we model a data table after the query we plan to execute



***

### Query 1


#### Create & Insert Operations for the `session_item` table

To query efficiently, we assign `sessionId` and `itemInSession` as the primary key. This will partition the table under `sessionId` and `itemInSession` 

In [23]:
# Create session_item table

query = "CREATE TABLE IF NOT EXISTS session_item"
query = query + "(sessionId bigint, itemInSession bigint, artist text, song text, length float, PRIMARY KEY ((sessionId, itemInSession), artist, song, length))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [24]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip the first row
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_item (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Assign column element to each column in the INSERT statement
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Query Statement

Get the artist, song title and song's length in the music app history that was heard during `sessionId = 338`, and `itemInSession  = 4`

In [25]:
# Add in the SELECT statement to verify the data was entered into the table

# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4

query = """ SELECT artist, song, length FROM session_item
            WHERE sessionId = 338 AND itemInSession = 4
        """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for rows in rows:
    print(rows.artist, rows.song, rows.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2

#### Create & Insert Operations for the `user_session` table

`userId` and `sessionId` are assigned as the primary key. The  table will be partitioned by same columns. We use itemInSession and song as clustering columns. This will sort the songs by itemInSession first, then the song title in alphabetical order and finally by artist name.

In [26]:
# Create user_session table

query = "CREATE TABLE IF NOT EXISTS user_session"
query = query + "(userId bigint, sessionId bigint, itemInSession bigint, artist text, song text, user text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [27]:
# Setup csv

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # Skip first row
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session(userId, sessionId, itemInSession, artist, song, user)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        # Assign column element to each column in the INSERT statement
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1] + " " + line[4]))

#### Query Statement

Get the name of the artist, song (sorted by itemInSession) and user (first and last name) for `userid = 10`, `sessionid = 182`

In [28]:
# Query 2: Get the name of artist, song (sorted by itemInSession) and user (first and last name)\
# WHERE userId = 10 AND sessionId = 182

query = "SELECT artist, song, user FROM user_session WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist, row.song, row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3

#### Create & Insert Operations for the `user_song` table

To achieve the query result we assign `song` as the primary key and partition by same column. This allows for efficient querying by WHERE clause based on song title. 

The `user` column is used as a clustering column to sort users in alphabetical order.

In [29]:
# Create user_song table

query = "CREATE TABLE IF NOT EXISTS user_song"
query = query + "(song text, user text, PRIMARY KEY(song))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [30]:
# Setup csv

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # Skip first row
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_song (song, user)"
        query = query + " VALUES(%s, %s)"
        # Assign column element to each column in the INSERT statement
        session.execute(query, (line[9], line[1] + " " + line[4]))

#### Query Statement

Get every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

In [33]:
# Query 3: Get every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

query = "SELECT user FROM user_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.user)

Tegan Levine


### Drop the tables before closing out the sessions

In [4]:
# Drop the table before closing out the sessions

drop_query_0 = "drop table session_item"
drop_query_1 = "drop table user_session"
drop_query_2 = "drop table user_song"
try:
    session.execute(drop_query_0)
    session.execute(drop_query_1)
    session.execute(drop_query_2)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()